# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

In [1]:
from typing import List, Tuple, Dict, Callable

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> List[Tuple]:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return the empty List (`[]`).


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

<a id="Helper Functions"></a>
## Helper Functions

<a id="Imports"></a>
### Imports

In [3]:
from itertools import permutations
from copy import deepcopy

<a id="get_cols"></a>
### get_cols

`get_cols` takes in a game board, and outputs the columns in list form to be processed in other functions. **Used by**: [get_dominated](#get_dominated)

***Inputs:***
* **game**: List[List[Tuple]]]: Current game board with each cell in (P1, P2) format

**returns** cols: List[List[Tuple]] of all columns in game board

In [4]:
def get_cols(game: List[List[Tuple]]): 
    cols = []
    
    for j in range(len(game[0])): #each col
        col = []    
        for i in range(len(game)): #each row
            col.append(game[i][j])
        cols.append(col)

    return cols

In [5]:
# Unit Tests
test_game = [
    [(1, 1), (2, 2)], 
    [(3, 3), (4, 4)]]
assert get_cols(test_game) == [[(1, 1), (3, 3)], [(2, 2), (4, 4)]]
test_game = [[(1, 1), (2, 2)]]
assert get_cols(test_game) == [[(1, 1)], [(2, 2)]]
test_game = [[(1,1)]]
assert get_cols(test_game) == [[(1,1)]]

<a id="get_rows"></a>
### get_rows

`get_rows` takes in a game board, and outputs the rows in list form to be processed in other functions. **Used by**: [get_dominated](#get_dominated)

***Inputs:***
* **game**: List[List[Tuple]]]: Current game board with each cell in (P1, P2) format

**returns** rows: List[List[Tuple]] of all rows in game board

In [6]:
def get_rows(game: List[List[Tuple]]): 
    rows = []
    for row in game: 
        rows.append(row)

    return rows

In [7]:
# Unit Tests / Assertions
test_game = [
    [(1, 1), (2, 2)], 
    [(3, 3), (4, 4)]]
assert get_rows(test_game) == [[(1, 1), (2, 2)], [(3, 3), (4, 4)]]
test_game = [[(1, 1), (2, 2)]]
assert get_rows(test_game) == [[(1, 1), (2, 2)]]
test_game = [[(1,1)]]
assert get_rows(test_game) == [[(1,1)]]

<a id="get_dominated"></a>
### get_dominated

`get_dominated` takes in a game board, and outputs the columns in list form to be processed in other functions. **Used by**: [solve_game](#solve_game)

***Inputs:***
* **game**: List[List[Tuple]]]: Current game board with each cell in (P1, P2) format
* **player**: int: Which player is acting, either 1 or 2
* **weak**: bool: Whether or not weakly dominated strategies can be allowed / removed

**returns** dominated_list: List[List[Tuple]] of all columns or rows to be deleted

In [8]:
def get_dominated(game, player: int, weak:bool=False): 
    if player == 2: 
        board = get_cols(game) #player 2 uses columns
    else: 
        board = game
    dominated_list, p = [], player-1
    for perm in permutations(board, 2): 
        dominant, one_greater = True, False
        len_ind = min(len(game), len(game[0]))
        for i in range(len_ind): #assumes square game
            if perm[0][i][p] > perm[1][i][p]: #last 0 index for P1
                one_greater = True
            elif (perm[0][i][p] == perm[1][i][p]) and weak == False: 
                dominant = False
            elif perm[0][i][p] < perm[1][i][p]: #change sign for weak / strong dominated (add equals)
                dominant = False
        if dominant == True and (perm[1] not in dominated_list) and one_greater == True: 
            dominated_list.append(perm[1])
    return dominated_list

In [9]:
# Unit Tests / Assertions
test_game = [
    [(1, 1), (2, 2)], 
    [(3, 3), (4, 4)]]
assert get_dominated(test_game, 1) == [[(1, 1), (2, 2)]]
assert get_dominated(test_game, 1, True) == [[(1, 1), (2, 2)]]
assert get_dominated(test_game, 2) == [[(1, 1), (3, 3)]]

<a id="get_index"></a>
### get_index

`get_index` takes in the game, a list of spots to test (individual, row, or column), and the player number to calculate the index of that spot. **Used by**: [remove](#remove)

***Inputs:***
* **game**: List[List[Tuple]]]: Current game board with each cell in (P1, P2) format
* **spot_list**: List[List[Tuple]]: List of spots (or rows or columns) to test
* **player**: Int: Player with current turn

**returns** index: List[List[Tuple]] of all columns in game board

In [10]:
def get_index(game: List[List[Tuple]], spot_list: List[List[Tuple]], player: int): 
    
    if player == 1: 
        test = get_rows(game)
    elif player == 2: 
        test = get_cols(game)

    index = test.index(spot_list)
    return index
    

In [11]:
# Unit Tests / Assertions
test_game = [
    [(1, 1), (2, 2)], 
    [(3, 3), (4, 4)]]
assert get_index(test_game, [(1, 1), (2,2)], 1) == 0
assert get_index(test_game, [(3, 3), (4, 4)], 1) == 1
assert get_index(test_game, [(1, 1), (3, 3)], 2) == 0

<a id="remove"></a>
### remove

`remove` takes in a game board, and outputs the columns in list form to be processed in other functions. **Used by**: [solve_game](#solve_game)

***Inputs:***
* **game**: List[List[Tuple]]]: Current game board with each cell in (P1, P2) format
* **del_list**: List[List[Tuple]]: List of spots (or rows or columns) to delete
* **player**: Int: Player with current turn

**returns** new_board: List[List[Tuple]] of new board after deletion(s)

In [12]:
def remove(game, del_list, player): 
    new_board = deepcopy(game)
    index = get_index(game, del_list, player)
    p = player - 1

    if player == 1: #delete row
        del new_board[index]
        
    if player == 2: #delete col
        for j in new_board: 
            del j[index]
                
    return new_board

In [13]:
# Unit Tests / Assertions
test_game = [
    [(1, 1), (2, 2)], 
    [(3, 3), (4, 4)]]
assert remove(test_game, [(1, 1), (2, 2)], 1) == [[(3, 3), (4, 4)]]
assert remove(test_game, [(3, 3), (4, 4)], 1) == [[(1, 1), (2, 2)]]
assert remove(test_game, [(1, 1), (3, 3)], 2) == [[(2, 2)], [(4, 4)]]

<a id="opp"></a>
### opp

`opp` takes in a player number, and returns the other player. This is used to swap players between turns. **Used by**: [solve_game](#solve_game)

***Inputs:***
* **player**: Int: Player with current turn

**returns** player: int, opposite player

In [14]:
def opp(player: int): 
    if player == 1: 
        return 2
    elif player == 2: 
        return 1
    else: 
        return 'Error!'

In [15]:
# Unit Tests / Assertions
assert opp(1) == 2
assert opp(2) == 1
assert opp(3) == 'Error!'

<a id="get_cartesian"></a>
### get_cartesian

`get_cartesian` takes in a game board and a nash equilibrium point, and outputs the cartesian indices of that nash point within the board. **Used by**: [check_nash](#check_nash)

***Inputs:***
* **game**: List[List[Tuple]]]: Current game board with each cell in (P1, P2) format
* **nash_point**: List[List[Tuple]]]: List of nash point in list form

**returns** (i,j): Tuple of cartesian index of nash point within game board

In [16]:
def get_cartesian(game: List[List[Tuple]], nash_point: Tuple): 
    for i, row in enumerate(game):
        for j, col in enumerate(row): 
            if game[i][j] == nash_point: 
                return (i,j)

In [17]:
# Unit Tests / Assertions
test_game = [
    [(1, 1), (2, 2)], 
    [(3, 3), (4, 4)]]
assert get_cartesian(test_game, (1, 1)) == (0, 0)
assert get_cartesian(test_game, (4, 4)) == (1, 1)
assert get_cartesian(test_game, (3, 3)) == (1, 0)

<a id="check_nash"></a>
### check_nash

`check_nash` takes in a game board, the next state, and the current list of nash equilibrium points. Verifies whether the next point is a nash point, then adds its index to the nash list. **Used by**: [solve_game](#solve_game)

***Inputs:***
* **game**: List[List[Tuple]]]: Current game board with each cell in (P1, P2) format
* **next_board**: List[List[Tuple]]]: Next game board, after deleting a dominated strategy
* **nash_list**: List[List[Tuple]] of all current Nash points

**returns** nash_list: List[[Tuple]] of all nash equilibrium point indices

In [18]:
def check_nash(game: List[List[Tuple]], next_board: List[List[Tuple]], nash_list: List[List[Tuple]]): 
    
    if (len(next_board) == 1 and len(next_board[0]) == 1 and len(next_board[0][0]) == 2): #nash
        nash_point = next_board[0][0]
        loc = get_cartesian(game, nash_point)
        if loc not in nash_list: 
            nash_list.append(loc)
    return nash_list

In [19]:
# Unit Tests / Assertions
test_game = [[(3, 3), (4, 4)]]
next_board = [[(4, 4)]]
assert check_nash(test_game, next_board, []) == [(0, 1)]
next_board = [[(3, 3)]]
assert check_nash(test_game, next_board, []) == [(0, 0)]
test_game = [
    [(1, 1), (2, 2)], 
    [(3, 3), (4, 4)]]
next_board = [[(3, 3)]]
assert check_nash(test_game, next_board, []) == [(1, 0)]

<a id="nash_print"></a>
### nash_print

`nash_print` takes in a game board and the nash list, and prints out the results in a neat format. Given this is a print function, unit tests / assertions are not necessary. 

***Inputs:***
* **game**: List[List[Tuple]]]: Current game board with each cell in (P1, P2) format
* **nash**: List[Tuple]: List of nash index points

**returns** print statements

In [20]:
def nash_print(game: List[List[Tuple]], nash: List[Tuple]): 
    print('Total Nash Equilibria: ', len(nash))
    for i in nash: 
        print('Player 1 chooses ', i[0], ', and Player 2 chooses ', i[1], '. Payoffs for (P1, P2): ', game[i[0]][i[1]])
    print()

<a id="next_move"></a>
### next_move

`next_move` takes in a current game board, initial game, dominated item list, player number, and nash list. Outputs next_board and new nash list (of indices). This is a combo function to reduce line count. **Used by**: [solve_game](#solve_game)

***Inputs:***
* **game**: List[List[Tuple]]]: Original game board with each cell in (P1, P2) format
* **board**: List[List[Tuple]]: Current game board
* **dom_item**: List of dominated item to remove
* **player**: Int player with current move
* **nash**: List of indices for nash equilibrium points

**returns** next_board and updated nash list

In [21]:
def next_move(board: List[List[Tuple]], dom_item: List[List[Tuple]], player: int, game: List[List[Tuple]], nash: List[Tuple]): 
    next_board = remove(board, dom_item, player)
    nash_list = deepcopy(nash)
    nash_list = check_nash(game, next_board, nash)

    return next_board, nash

In [22]:
# Unit Tests / Assertions
test_game = [
    [(1, 1), (2, 2)], 
    [(3, 3), (4, 4)]]
board = [[(3, 3), (4, 4)]]
next_board, nash = next_move(board, [(3, 3)], 2, test_game, [])
assert next_board == [[(4, 4)]]
assert len(next_board[0]) == 1
assert nash == [(1, 1)]

<a id="solve_game"></a>
## solve_game

`solve_game` is the main driving function. This takes in a game board and a bool value on whether weakly dominated strategies can be eliminated. No assertions are needed, as the following problems will prove/disprove the algorithm. 

***Inputs:***
* **game**: List[List[Tuple]]]: Current game board with each cell in (P1, P2) format

**returns** nash: List[Tuple] of all nash equilibrium indices, referencing the original game board

In [23]:
def solve_game(game: List[List[Tuple]], weak:bool=False) -> List[Tuple]:
    nash = [] #init nash list
    for i in range(1, 3): # start with player 1, then 2
        player, cur_board, run_list, iteration = i, deepcopy(game), [], 0 #init
        run_list.append([game, get_dominated(game, player, weak), player]) #init
        while run_list and iteration < 10: 
            cur_iter = run_list[0]
            board, dom_list, player = cur_iter[0], cur_iter[1], cur_iter[2]
            if len(dom_list) == 0 and run_list: #no more dominated strategies
                run_list.pop(0)
                i+= 1
                continue
            next_board, nash = next_move(board, dom_list[0], player, game, nash)
            run_list.append([next_board, get_dominated(next_board, opp(player), weak), opp(player)])
            if len(dom_list) > 1: 
                dom_list.pop(0)
                run_list.append([board, dom_list, player])
            run_list.pop(0)
            iteration +=1
    return nash

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices (a list of them).

## Before you code...

Solve the following game by hand using SEDS and weakly dominated strategies. 
The game has three (pure) Nash Equilibriums. 
You should find all of them.
This will help you think about what you need to implement to make the algorithm work.
**Hint**: You will need State Space Search from Module 1 and SEDS from Module 5 to get the full algorithm to work.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

### Test Game - By Hand

**Solution #1**: 

Starting with Player 1, strategy 2 weakly dominates strategy 1, so we can remove strategy 1: 
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|2  | 2/2 | 3/3 | 0/2 |

For Player 2, strategy 1 weakly dominates strategy 2, so we can eliminate it: 
| Player 1 / Player 2  | 0 | 1 |
|----|----|----|
|0  | 1/0 | 3/1 |
|2  | 2/2 | 3/3 |

For Player 1, strategy 2 weakly dominates strategy 0
| Player 1 / Player 2  | 0 | 1 |
|----|----|----|
|2  | 2/2 | 3/3 |

For Player 2, strategy 1 weakly dominates 0, leaving us with the first nash equilibrum: Player 1 chooses 2, Player 2 chooses 1, for payoffs of 2/2
| Player 1 / Player 2  | 1 |
|----|----|
|2  | 3/3 |

-----

**Solution #2**
Player 2, Strategy 2 weakly dominates 0: 
| Player 1 / Player 2  | 1 | 2 |
|----|----|----|
|0  | 3/1 | 1/1 |
|1  | 3/0 | 0/1 |
|2  | 3/3 | 0/2 |

Player 1: Strategy 0 weakly dominates 2: 
| Player 1 / Player 2  | 1 | 2 |
|----|----|----|
|0  | 3/1 | 1/1 |
|1  | 3/0 | 0/1 |

Player 2: Strategy 2 weakly dominates 1: 
| Player 1 / Player 2  | 2 |
|----|----|
|0  | 1/1 |
|1  | 0/1 |

Player 1: Strategy 0 weakly dominates 1, leaving us with second nash equilibrium: Player 1 chooses 0, Player 2 chooses 2, for payoffs of 1/1
| Player 1 / Player 2  | 2 |
|----|----|
|0  | 1/1 |

**Solution #3** 
*This solution cannot be found with SEDS alone - we also need to use cell inspection*
Player 1: Strategy 0 weakly dominates 1: 
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|2  | 2/2 | 3/3 | 0/2 |

Player 2: Strategy 2 weakly dominates 0
| Player 1 / Player 2  | 1 | 2 |
|----|----|----|
|0  | 3/1 | 1/1 |
|2  | 3/3 | 0/2 |

Player 1: Strategy 0 weakly dominates 2, leaving us with: 
| Player 1 / Player 2  | 1 | 2 |
|----|----|----|
|0  | 3/1 | 1/1 |

Player 2 is the next to go, but since both options result in the same payout (1), we have to resolve this by inspection. If Player 1 chooses 0, Player 2's best option is to choose 1, resulting in payouts of (3,1). 
| Player 1 / Player 2  | 1 |
|----|----|
|0  | 3/1 |

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/4 | 3/4 | 2/3 |
|2  | 2/2 | 5/5 | 3/2 |

**Solution:**
Player 1: 2 dominates 0
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|1  | 1/4 | 3/4 | 2/3 |
|2  | 2/2 | 5/5 | 3/2 |

Player 2: 1 dominates 2
| Player 1 / Player 2  | 0 | 1 |
|----|----|----|
|1  | 1/4 | 3/4 |
|2  | 2/2 | 5/5 |

Player 1: 2 dominates 1
| Player 1 / Player 2  | 0 | 1 |
|----|----|----|
|2  | 2/2 | 5/5 |

Player 2: 1 dominates 0 --> Nash = 2, 1 indices
| Player 1 / Player 2  | 1 |
|----|----|
|2  | 5/5 |

In [24]:
test_game_1 = [
    [(1, 0), (3, 1), (1, 1)],
    [(1, 4), (3, 4), (2, 3)],
    [(2, 2), (5, 5), (3, 2)]]

solution = solve_game(test_game_1)
print(solution)
nash_print(test_game_1, solution)

[(2, 1)]
Total Nash Equilibria:  1
Player 1 chooses  2 , and Player 2 chooses  1 . Payoffs for (P1, P2):  (5, 5)



In [25]:
assert solution == [(2, 1)] # insert your solution from above.

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 9/4 | 3/1 | 1/9 |
|1  | 1/4 | 3/7 | 3/3 |
|2  | 2/2 | 5/5 | 3/2 |

**Solution:** 
*Using Weakly Dominated strategies*
Player 1: 2 dominates 1
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 9/4 | 3/1 | 1/9 |
|2  | 2/2 | 5/5 | 3/2 |

Player 2: 2 dominates 0
| Player 1 / Player 2  | 1 | 2 |
|----|----|----|
|0  | 3/1 | 1/9 |
|2  | 5/5 | 3/2 |

Player 1: 2 dominates 0
| Player 1 / Player 2  | 1 | 2 |
|----|----|----|
|2  | 5/5 | 3/2 |

Player 2: 1 dominates 2
| Player 1 / Player 2  | 1 |
|----|----|
|2  | 5/5 |

In [26]:
test_game_2 = [
    [(9, 4), (3, 1), (1, 9)],
    [(1, 4), (3, 7), (3, 3)],
    [(2, 2), (5, 5), (3, 2)]]

strong_solution = solve_game(test_game_2)
weak_solution = solve_game(test_game_2, weak=True)

nash_print(test_game_2, strong_solution)
nash_print(test_game_2, weak_solution)

Total Nash Equilibria:  0

Total Nash Equilibria:  1
Player 1 chooses  2 , and Player 2 chooses  1 . Payoffs for (P1, P2):  (5, 5)



In [27]:
assert strong_solution == []
assert weak_solution == [(2, 1)] # insert your solution from above.

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 9/4 | 4/9 | 9/4 |
|1  | 1/4 | 5/2 | 2/5 |
|2  | 2/2 | 5/1 | 3/2 |

**Solution:** None
Player 1 - No dominated strategies
Player 2 - No dominated strategies

In [28]:
test_game_3 = [
    [(9, 4), (4, 9), (9, 4)],
    [(1, 4), (5, 2), (2, 5)],
    [(2, 2), (5, 1), (3, 2)]]

strong_solution = solve_game( test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

nash_print(test_game_3, strong_solution)
nash_print(test_game_3, weak_solution)

Total Nash Equilibria:  0

Total Nash Equilibria:  0



In [29]:
assert strong_solution == []
assert weak_solution == []

### Test Game 4. Multiple Equilibria

You solve the following game by hand, above.
Now use your code to solve it.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions:** (copy from above)

In [30]:
test_game_4 = [
[(1, 0), (3, 1), (1, 1)],
[(1, 1), (3, 0), (0, 3)],
[(2, 2), (3, 3), (0, 2)]]

strong_solution = solve_game(test_game_4)
weak_solution = solve_game( test_game_4, weak=True)

In [31]:
nash_print(test_game_4, strong_solution)
nash_print(test_game_4, weak_solution)

Total Nash Equilibria:  0

Total Nash Equilibria:  2
Player 1 chooses  2 , and Player 2 chooses  1 . Payoffs for (P1, P2):  (3, 3)
Player 1 chooses  0 , and Player 2 chooses  2 . Payoffs for (P1, P2):  (1, 1)



In [32]:
assert strong_solution == []
assert weak_solution == [(2, 1), (0, 2)] # put solution here

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.